In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=2f60f6a047c4db5776cd90c49751eaa9b5257af4d9d7debd4aa8a060f3dfe577
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [2]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7niv12md/kobert-tokenizer_ec7a7579a9d5448eb43df753e08e0934
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7niv12md/kobert-tokenizer_ec7a7579a9d5448eb43df753e08e0934
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=8300e500e003badddedd227d8975364a3e34daabff6870db36dfdf0939795dcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-2owa1cub/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
!pip3 install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# GPU 사용 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
# ●
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
# 파라미터 세팅
max_len = 16
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [9]:
import pandas as pd 

path = './'
train_data = pd.read_csv(path + 'review_dataset.csv',encoding='utf-8-sig')

train_dataset = []
for sen, label in zip(train_data['content'], train_data['메뉴명']):
    data_train = []
    data_train.append(sen)
    data_train.append(str(label))

    train_dataset.append(data_train)

In [10]:
# 훈련, 테스트 데이터셋 분리 
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(train_dataset, test_size=0.25, random_state=0)

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=True
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }
        # print(torch.tensor(inputs['input_ids']).shape)
        # print(torch.tensor(inputs['attention_mask']).shape)
        # print(torch.tensor(inputs['token_type_ids']).shape)

        item['labels'] = int(self.dataset[idx][1])
        # item['labels'] = self.dataset[idx][1]

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']


    def __len__(self):
        return len(self.dataset)


In [12]:
# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
train_dataset[0]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(array([   2, 4635,    3, 1875,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1]),
 array([16]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4)

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [15]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [16]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


In [17]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [19]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # token_ids = token_ids.long()
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # print(label)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):
        token_ids = token_ids.long().to(device)
        # token_ids = token_ids.long()
        segment_ids = segment_ids.long().to(device)
        # segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


<ipython-input-19-00a7fd27a515>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6116536855697632 train acc 0.1875
epoch 1 train acc 0.3395061728395061


<ipython-input-19-00a7fd27a515>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):


  0%|          | 0/6 [00:00<?, ?it/s]

...

epoch 29 test acc 0.8668154761904762


  0%|          | 0/18 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.10119611024856567 train acc 0.9375
epoch 30 train acc 0.8670910493827161


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 30 test acc 0.8668154761904762


In [20]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, tokenizer, max_len, True, True)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        # token_ids = token_ids.long()
        # segment_ids = segment_ids.long()

        valid_length= valid_length
        label = label.long().to(device)
        # label = label.long()

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            print(i)
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("아아")
            elif np.argmax(logits) == 1:
                test_eval.append("따아")
            elif np.argmax(logits) == 2:
                test_eval.append("콜드브루")
            elif np.argmax(logits) == 3:
                test_eval.append("아이스 카라멜 마끼아또")
            elif np.argmax(logits) == 4:
                test_eval.append("카라멜마끼아또")    
            else:
                test_eval.append("None")
        print(test_eval)

In [21]:
predict("시원한 음료 먹고싶어")
predict("단 거 먹고싶어")
predict("카라멜")
predict("단 맛")
predict("콜드 브루")
predict("따뜻한")
predict("여름")
predict("카라멜의 향긋한 향이 풍부한 음료입니다")

tensor([ 3.6172, -1.7617, -3.1304,  2.1602, -0.7219], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']
tensor([ 3.5992, -1.4697, -2.7164,  2.4108, -1.8522], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']
tensor([-1.8217, -1.6470, -1.5568, -1.5533,  6.6857], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']
tensor([ 3.5992, -1.4697, -2.7164,  2.4108, -1.8522], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']
tensor([-1.8451, -1.1792,  5.8543, -1.2881, -1.5308], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['콜드브루']
tensor([-1.3781,  5.6924, -1.3281, -1.4186, -1.7829], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['따아']
tensor([ 3.7505, -1.3920, -2.8690,  2.3619, -1.8412], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']
tensor([-1.8217, -1.6470, -1.5568, -1.5533,  6.6857], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']


In [22]:
predict("너무 더운데 시원한 음료 마시고 싶어")

tensor([ 3.8993, -1.3964, -2.9970,  2.1546, -1.6809], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']


In [23]:
predict("아아 추가해줘")

tensor([ 3.0895, -1.3501, -2.7331,  3.0157, -1.6328], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['아아']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
predict("카라멜 들어간 거")

tensor([-1.8217, -1.6470, -1.5568, -1.5533,  6.6857], device='cuda:0',
       grad_fn=<UnbindBackward0>)
['카라멜마끼아또']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
